In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, cross_validate

In [26]:
"""
def calc_scores(y_test, y_pred):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    f2_score = (0 if all(y_pred == 0) else metrics.fbeta_score(y_test, y_pred, beta=2))
    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))

    return accuracy, f2_score, dmc_score, confusion_matrix
"""

'\ndef calc_scores(y_test, y_pred):\n    accuracy = metrics.accuracy_score(y_test, y_pred)\n    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)\n    f2_score = (0 if all(y_pred == 0) else metrics.fbeta_score(y_test, y_pred, beta=2))\n    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))\n\n    return accuracy, f2_score, dmc_score, confusion_matrix\n'

In [27]:
def own_scorer(estimator, X_val, ground_truth):
    prediction = estimator.predict(X_val)
    confusion_matrix = metrics.confusion_matrix(ground_truth, prediction)
    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))
    return dmc_score

In [28]:
def own_scorer_normalized(estimator, X_val, ground_truth):
    prediction = estimator.predict(X_val)
    confusion_matrix = metrics.confusion_matrix(ground_truth, prediction)
    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))
    return dmc_score/len(ground_truth)

In [29]:
def own_f2_score(estimator, X_val, ground_truth):
    prediction = estimator.predict(X_val)
    return 0 if all(prediction == 0) else metrics.fbeta_score(ground_truth, prediction, beta=2)

In [30]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [31]:
def own_cross_validation(classifier, data, cv=5):
    k = KFold(n_splits=cv)
    splits = k.get_n_splits(data)
    print(splits.shape)

## Testing your Classifier by multiple k fold cross validates and shuffles. Currently only sklearn familiar classifiers are allowed

In [32]:
def test_classification(classifier, df_train, df_val, predict_proba=True, ret_dataframe=True, ):
    result = dict()
    dmc_scores = list()
    dmc_scores_norm = list()
    res_dataframe = df_val.copy()
    y_train = df_train.fraud
    x_train = df_train.drop(columns=['fraud'])
    y_val = df_val.fraud
    x_val = df_val.drop(columns=['fraud'])
    
    classifier.fit(x_train, y_train)
    
    dmc_score = own_scorer(classifier, x_val, y_val)
    dmc_norm = own_scorer_normalized(classifier, x_val, y_val)
    res_dataframe['prediction'] = classifier.predict(x_val)
    
    if predict_proba:
        res_dataframe['probablity'] = [round(max(x),3) for x in classifier.predict_proba(x_val)]
    # Cross Validation
    scorings = {"DMC" : own_scorer, "DMC_Norm" : own_scorer_normalized}

    
    x_train_complete = x_train.append(x_val)
    y_train_complete = y_train.append(y_val)
    cross_validation = cross_validate(classifier, x_train_complete,y_train_complete,scoring=scorings, cv=5)
    cv_dmc_mean = sum(cross_validation['test_DMC'])/len(cross_validation['test_DMC'])
    cv_dmc_norm_mean = sum(cross_validation['test_DMC_Norm'])/len(cross_validation['test_DMC_Norm'])
    
    result = {"dmc_score" : dmc_score, "dmc_score_norm" : dmc_norm, "cv_dmc_score": cv_dmc_mean,"cv_dmc_score_norm":cv_dmc_norm_mean}
    if ret_dataframe:
        result['dataframe'] = res_dataframe
    print("Results Fix Split: \nDMC Score: {}  ---  Normalized DMC Score: {}, \n\nResults Cross Validation: \nDMC Score: {}  ---  Normalized DMC Score: {} ".format(dmc_score, dmc_norm, cv_dmc_mean,cv_dmc_norm_mean))
    return result
    

## Adding additional features by calling this on the specific Dataframe

In [33]:
def add_new_features(dataframe):
    dataframe['totalScannedItems'] = dataframe['scannedLineItemsPerSecond'] * dataframe['totalScanTimeInSeconds']
    return dataframe

## Scale dataframe with Sklean Scaler (Please scale only model input "train_x")

In [34]:
def scale_dataframe(df, scaler):
    df_tmp = pd.DataFrame()
    tmp_data = scaler.transform(df[df.columns])
    df_tmp[df.columns] = pd.DataFrame(tmp_data)
    return df_tmp.copy()

## Find nearest neigbour to test sample

In [35]:
def find_nearest_neighbor(row_scaled, dataset_scaled):  
    diffs = [np.sum((row_scaled[0] - ds_row)**2) for ds_row in dataset_scaled]
    idx = np.argmin(diffs)[0]
    return idx, diffs[idx]

In [36]:
def best_classifier_for_sample(idx, validation_set):
    ground_truth = validation_set.iloc[idx].fraud
    
    # Both classifier predicted the calue correctly
    if (validation_set.iloc[idx].lsvc_predict == ground_truth) and (validation_set.iloc[idx].xgb_predict == ground_truth):
        if validation_set.iloc[idx].lsvc_proba > validation_set.iloc[idx].xgb_proba:
            return "lsvc"
        else:
            return "xgboost"
    # lsvc predicted correctly
    elif (validation_set.iloc[idx].lsvc_predict == ground_truth) and (validation_set.iloc[idx].xgb_predict != ground_truth):
        return "lsvc"
    
    # xgboost predicted correcltly
    elif (validation_set.iloc[idx].lsvc_predict != ground_truth) and (validation_set.iloc[idx].xgb_predict == ground_truth):
        return "xgboost"
    
    # If No classifier predicted the knn correct, None is returned
    else: 
        return None
    
    

## Transformer Class

In [37]:
class DataTransformer:
    """
    for scaling, data transformations (new features, one-hot encoding, categorical, ...)
    """
    
    def __init__(self, scaler):
        self.scaler = scaler

    def fit_scaler(self, df):
        df_tmp = df.copy()
        self.scaler.fit(df_tmp.astype(np.float64))
        return self
        
    def apply_scaler(self, df):
        df_temp = df.copy()
        return pd.DataFrame(self.scaler.transform(df_temp),df_temp.index, df_temp.columns)
    
    def inverse_scale(self, df):
        df_tmp = df.copy()
        return pd.DataFrame(self.scaler.inverse_transform(df_tmp), df_tmp.index, df_tmp.columns)
    
    def add_features(self,df):
        #TODO: Choose relevant features
        df_tmp = df.copy()
        df_tmp['totalScannedLineItems'] = df_tmp['scannedLineItemsPerSecond'] * df_tmp['totalScanTimeInSeconds']
        #df['avgTimePerScan'] = 1/ df['scannedLineItemsPerSecond']
        #df['avgValuePerScan'] = df['avgTimePerScan'] * df['valuePerSecond']
        #df['withoutRegisPerPosition'] = df['scansWithoutRegistration'] / df['totalScannedLineItems'] #equivalent to lineItemVoidsPerPosition?
        #df['quantiModPerPosition'] = df['quantityModifications'] / df['totalScannedLineItems']
        #df['lineItemVoidsPerTotal'] = df['lineItemVoids'] / df['grandTotal']
        #df['withoutRegisPerTotal'] = df['scansWithoutRegistration'] / df['grandTotal']
        #df['quantiModPerTotal'] = df['quantityModifications'] / df['grandTotal']
        #df['lineItemVoidsPerTime'] = df['lineItemVoids'] / df['totalScanTimeInSeconds']
        #df['withoutRegisPerTime'] = df['scansWithoutRegistration'] / df['totalScanTimeInSeconds']
        #df['quantiModPerTime'] = df['quantityModifications'] / df['totalScanTimeInSeconds']
        #df['valuePerScannedLineItem'] = df['valuePerSecond'] / df['scannedLineItemsPerSecond']
        return df_tmp
    
    def transform(self, df):
        """
        All in one: Apply all transform methods
            1.) addFeatures
            2.) apply_scaler
        """
        df_tmp = df.copy()
        return self.apply_scaler(self.add_features(df_tmp))
    


## KNN class

In [38]:
class KNNLookup():

    def __init__(self, knn_data):

        #self.knn = NearestNeighbors(n_neighbors=1)
        #self.knn.fit(knn_data.values.tolist())

        self.tree = KDTree(knn_data)

    def refit(self, knn_data):
        self.tree = KDTree(knn_data)

    def find_nearest_neighbor(self, row_scaled, dataset_scaled):
        diffs = [np.sum((row_scaled - ds_row) ** 2) for idx, ds_row in dataset_scaled.iterrows()]
        idx = np.argmin(diffs)
        return idx, diffs[idx]

    def find_nearest_neighbor2(self, row_scaled, dataset_scaled):
        dist, ind = self.tree.query([row_scaled.values], k=1)
        return np.ravel(ind)[0], np.ravel(dist)[0]

## Plot results

In [39]:
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
def plot_results_ssl(result_dict):
    # recall = TP / (TP + FN)
    # precision = TP / (TP + FP)

    dmc_scores_val = {}
    #dmc_scores_val = dict.fromkeys(['lin_svg', 'xgboost', 'own_classifier'])
    dmc_scores_train = {}
    precision_val = {}
    precision_train = {}
    recall_val = {}
    recall_train = {}

    for iteration_number, elem in enumerate(res):
        iteration_dict = elem
        #print("# Iteration: ", iteration_number, "\n# Value: ", iteration_dict)
        for classifier, results in iteration_dict.items():
            #print("\t## Classifier: ", classifier, "\n\t## Results: ", results)
            for set_name, result in results.items():
                conf_matrix = result['conf_matrix']
                tp = conf_matrix[0][0]
                fp = conf_matrix[0][1]
                fn = conf_matrix[1][0]
                tn = conf_matrix[1][1]

                recall = tp / (tp + fn)
                precision = tp / (tp + fp)

                if set_name == "val":
                    if classifier in dmc_scores_val:
                        dmc_scores_val[classifier].append(result['dmc_score'])
                    else:
                        dmc_scores_val[classifier] = [result['dmc_score']]
                    if classifier in recall_val:
                        recall_val[classifier].append(recall)
                    else:
                        recall_val[classifier] = [recall]
                    if classifier in precision_val:
                        precision_val[classifier].append(precision)
                    else:
                        precision_val[classifier] = [precision]
                else:
                    if classifier in dmc_scores_train:
                        dmc_scores_train[classifier].append(result['dmc_score'])
                    else:
                        dmc_scores_train[classifier] = [result['dmc_score']]
                    if classifier in recall_train:
                        recall_train[classifier].append(recall)
                    else:
                        recall_train[classifier] = [recall]
                    if classifier in precision_train:
                        precision_train[classifier].append(precision)
                    else:
                        precision_train[classifier] = [precision]
                #print("\t\t### Set: ", set_name, "\n\t\t### Result: ", result)
                #print("TP: ", tp, "\tFP: ", fp, "\tFN: ", fn, "\tTN: ", tn)


    #print(dmc_scores_val)
    #print(dmc_scores_train)
    #print(precision_val)
    #print(precision_train)
    #print(recall_val)
    #print(recall_train)
    
    plt.figure(num=1, figsize=(7,7))
    
    # ----------------- DMC Score on train set ---------------------------------
    
    ax1 = plt.subplot(311)
    ax1.set_ylim(-80, 520)
    
    x_values = np.arange(1, len(dmc_scores_train['lin_svc']) + 1, 1)
    x_new = np.linspace(x_values.min(), x_values.max(), 300)
    
    ax1.set_xticks(x_values)
   
    spl = make_interp_spline(x_values, dmc_scores_train['lin_svc'], k=3) #BSpline object
    smooth_lin_svc = spl(x_new)
    ax1.plot(x_new, smooth_lin_svc, label='lin_svc', color='r') # Entwicklung der Lin SVC
    
    spl = make_interp_spline(x_values, dmc_scores_train['xgboost'], k=3) 
    smooth_xgboost = spl(x_new)
    ax1.plot(x_new, smooth_xgboost, label='xgboost', color='b') # Entwicklung des XGBoostings
    
    spl = make_interp_spline(x_values, dmc_scores_train['own_classifier'], k=3) 
    smooth_own_classifier = spl(x_new)
    ax1.plot(x_new, smooth_own_classifier, label='own_classifier', color='g') # Entwicklung des eigenen Klassifizierers
    
    ax1.set_title('DMC Score on the train set')
    ax1.set_ylabel('Score')
    ax1.set_xlabel('Iteration')
    ax1.legend()

    # ----------------- Precision on train set ---------------------------------
    
    ax2 = plt.subplot(312)
    
    x_values = np.arange(1, len(precision_train['lin_svc']) + 1, 1)
    x_new = np.linspace(x_values.min(), x_values.max(), 300)
    
    ax2.set_xticks(x_values)
    
    spl = make_interp_spline(x_values, precision_train['lin_svc'], k=3) 
    smooth_lin_svc = spl(x_new)
    ax2.plot(x_new, smooth_lin_svc, label='lin_svc', color='r')# Entwicklung der Lin SVC
    
    spl = make_interp_spline(x_values, precision_train['xgboost'], k=3) 
    smooth_xgboost = spl(x_new)
    ax2.plot(x_new, smooth_xgboost, label='xgboost', color='b')# Entwicklung des XGBoostings
    
    spl = make_interp_spline(x_values, precision_train['own_classifier'], k=3) 
    smooth_own_classifier = spl(x_new)
    ax2.plot(x_new, smooth_own_classifier, label='own_classifier', color='g')# Entwicklung des eigenen Klassifizierers
    ax2.set_title('Precision on the train set')
    ax2.set_ylabel('Precision')
    ax2.set_xlabel('Iteration')
    ax2.legend()
    
    # ----------------- Recall on train set ---------------------------------

    ax3 = plt.subplot(313)
    
    x_values = np.arange(1, len(recall_train['lin_svc']) + 1, 1)
    x_new = np.linspace(x_values.min(), x_values.max(), 300)
    
    ax3.set_xticks(x_values)
    
    spl = make_interp_spline(x_values, recall_train['lin_svc'], k=3) 
    smooth_lin_svc = spl(x_new)
    ax3.plot(x_new, smooth_lin_svc, label='lin_svc', color='r')# Entwicklung der Lin SVC
    
    spl = make_interp_spline(x_values, recall_train['xgboost'], k=3) 
    smooth_xgboost = spl(x_new)
    ax3.plot(x_new, smooth_xgboost, label='xgboost', color='b')# Entwicklung des XGBoostings
    
    spl = make_interp_spline(x_values, recall_train['own_classifier'], k=3) 
    smooth_own_classifier = spl(x_new)
    ax3.plot(x_new, smooth_own_classifier, label='own_classifier', color='g')# Entwicklung des eigenen Klassifizierers
    ax3.set_title('Recall on the train set')
    ax3.set_ylabel('Recall')
    ax3.set_xlabel('Iteration')
    ax3.legend()
    plt.subplots_adjust(left=0.0, right=1.0, bottom=1.8, top=3.5)

    plt.figure(2, figsize=(7,7))

    # ----------------- DMC Score on val set ---------------------------------

    ax4 = plt.subplot(311)
    ax4.set_ylim(-80, 115)
    
    x_values = np.arange(1, len(dmc_scores_val['lin_svc']) + 1, 1)
    x_new = np.linspace(x_values.min(), x_values.max(), 300)
    
    ax4.set_xticks(x_values)
    
    spl = make_interp_spline(x_values, dmc_scores_val['lin_svc'], k=3) 
    smooth_lin_svc = spl(x_new)
    ax4.plot(x_new, smooth_lin_svc, label='lin_svc', color='r')# Entwicklung der Lin SVC
    
    spl = make_interp_spline(x_values, dmc_scores_val['xgboost'], k=3) 
    smooth_xgboost = spl(x_new)
    ax4.plot(x_new, smooth_xgboost, label='xgboost', color='b')# Entwicklung des XGBoostings
    
    spl = make_interp_spline(x_values, dmc_scores_val['own_classifier'], k=3) 
    smooth_own_classifier = spl(x_new)
    ax4.plot(x_new, smooth_own_classifier, label='own_classifier', color='g')# Entwicklung des eigenen Klassifizierers
    ax4.set_title('DMC Score on the val set')
    ax4.set_ylabel('Score')
    ax4.set_xlabel('Iteration')
    ax4.legend()

    # ----------------- Precision on val set ---------------------------------
    
    ax5 = plt.subplot(312)
    
    x_values = np.arange(1, len(precision_val['lin_svc']) + 1, 1)
    x_new = np.linspace(x_values.min(), x_values.max(), 300)
    
    ax5.set_xticks(x_values)
    
    spl = make_interp_spline(x_values, precision_val['lin_svc'], k=3) 
    smooth_lin_svc = spl(x_new)    
    ax5.plot(x_new, smooth_lin_svc, label='lin_svc', color='r')# Entwicklung der Lin SVC
    
    spl = make_interp_spline(x_values, precision_val['xgboost'], k=3) 
    smooth_xgboost = spl(x_new)
    ax5.plot(x_new, smooth_xgboost, label='xgboost', color='b')# Entwicklung des XGBoostings
    
    spl = make_interp_spline(x_values, precision_train['own_classifier'], k=3) 
    smooth_own_classifier = spl(x_new)
    ax5.plot(x_new, smooth_own_classifier, label='own_classifier', color='g')# Entwicklung des eigenen Klassifizierers
    ax5.set_title('Precision on the val set')
    ax5.set_ylabel('Precision')
    ax5.set_xlabel('Iteration')
    ax5.legend()
    
    # ----------------- Recall on train set ---------------------------------

    ax6 = plt.subplot(313)
    
    x_values = np.arange(1, len(recall_val['lin_svc']) + 1, 1)
    x_new = np.linspace(x_values.min(), x_values.max(), 300)
    
    ax6.set_xticks(x_values)
    
    spl = make_interp_spline(x_values, recall_val['lin_svc'], k=3) 
    smooth_lin_svc = spl(x_new)
    ax6.plot(x_new, smooth_lin_svc, label='lin_svc', color='r')# Entwicklung der Lin SVC
    
    spl = make_interp_spline(x_values, recall_val['xgboost'], k=3) 
    smooth_xgboost = spl(x_new)
    ax6.plot(x_new, smooth_xgboost, label='xgboost', color='b')# Entwicklung des XGBoostings
    
    spl = make_interp_spline(x_values, recall_val['own_classifier'], k=3) 
    smooth_own_classifier = spl(x_new)
    ax6.plot(x_new, smooth_own_classifier, label='own_classifier', color='g')# Entwicklung des eigenen Klassifizierers
    ax6.set_title('Recall on the val set')
    ax6.set_ylabel('Recall')
    ax6.set_xlabel('Iteration')
    ax6.legend()

    plt.subplots_adjust(left=0.0, right=1.0, bottom=1.8, top=3.5)
    plt.show()

## Cross Validation for own classifier with ssl

In [40]:
def own_classifier_cross_validation(train_data_Xy_unscaled,test_dataset_X_unscaled, n_pltest_samples, cv, transformer):
    k = KFold(n_splits=cv)
    data_X_unscaled = train_data_Xy_unscaled.copy().drop(columns=['fraud'])
    data_y_unscaled = train_data_Xy_unscaled.fraud
    test_dataset = test_dataset_X_unscaled.copy()
    accs, dmcs, conf_mats = [],[],[]
    for train_index, test_index in k.split(data_X_unscaled):
        
        train_data_Xy_unscaled = data_X_unscaled.copy().iloc[train_index]
        train_data_X_unscaled = data_X_unscaled.copy().iloc[train_index]
        train_data_X_scaled = transformer.apply_scaler(train_data_X_unscaled)
        train_data_y = data_y_unscaled.copy().iloc[train_index]
               
        test_dataset_X_unscaled = test_dataset.copy().sample(frac=1)        
        testbatch_X_df_unscaled = test_dataset_X_unscaled.iloc[:n_pltest_samples].copy().reset_index(drop=True)
        testbatch_X_df_scaled = transformer.apply_scaler(testbatch_X_df_unscaled)
        
        pltrain_X_unscaled = train_data_X_unscaled.copy()
        pltrain_y = train_data_y.copy()
        
        pltrain_X_unscaled, pltrain_y = get_extended_pltrain_for_batch_without_knn(testbatch_X_df_unscaled, train_data_X_unscaled, train_data_y, transformer)
        
        linear_svc = get_classifier("svc")
        pltrain_X_scaled = transformer.apply_scaler(pltrain_X_unscaled)
        linear_svc.fit(pltrain_X_scaled.values, pltrain_y.values)
        
        xgboost = get_classifier("xgb")
        xgboost.fit(pltrain_X_unscaled.values, pltrain_y.values)
              
        
        validation_X_unscaled = data_X_unscaled.copy().iloc[test_index]
        validation_y = data_y_unscaled.copy().iloc[test_index]
        
        val_preds = classify_without_knn(xgboost, linear_svc, validation_X_unscaled, transformer)
        acc, dmc, conf_mat = calc_scores(validation_y, val_preds)
        accs.append(acc)
        dmcs.append(dmc)
        conf_mats.append(conf_mat)
        
    print("DMC Mean: {} --- DMC Sum: {}".format(np.mean(dmcs), np.sum(dmcs)))
    return np.mean(dmcs), np.sum(dmcs)

In [41]:
def create_nn_train_data(train_data_Xy_unscaled, cv, transformer):
    k = KFold(n_splits=cv)
    data_X_unscaled = train_data_Xy_unscaled.copy().drop(columns=['fraud'])
    data_y_unscaled = train_data_Xy_unscaled.fraud
    #test_dataset = test_dataset_X_unscaled.copy()
    
    predicted_val_df = pd.DataFrame()
    counter = 1
    for train_index, test_index in k.split(data_X_unscaled):
        print(counter, ". Split")
        train_data_Xy_unscaled = data_X_unscaled.copy().iloc[train_index]
        
        train_data_X_unscaled = data_X_unscaled.copy().iloc[train_index]
        train_data_X_scaled = transformer.apply_scaler(train_data_X_unscaled)
        train_data_y = data_y_unscaled.copy().iloc[train_index]
               
        validation_X_unscaled = data_X_unscaled.copy().iloc[test_index]
        validation_X_scaled = transformer.apply_scaler(validation_X_unscaled)
        validation_y = data_y_unscaled.copy().iloc[test_index]
        
        val_batch_df = validation_X_unscaled.copy()
                        
        linear_svc = get_classifier("svc")
        linear_svc.fit(train_data_X_scaled.values, train_data_y.values)
        
        xgboost = get_classifier("xgb")
        xgboost.fit(train_data_X_unscaled.values, train_data_y.values)
        
        lsvc_predict = linear_svc.predict(validation_X_scaled.values)
        lsvc_proba = linear_svc.predict_proba(validation_X_scaled.values)
        
        xgb_predict = xgboost.predict(validation_X_unscaled.values)
        xgb_proba = xgboost.predict_proba(validation_X_unscaled.values)
        
        val_batch_df['lsvc_predict'] = lsvc_predict     
        val_batch_df['lsvc_proba'] = [round(max(x),3) for x in lsvc_proba]
        val_batch_df['xgb_predict'] = xgb_predict     
        val_batch_df['xgb_proba'] = [round(max(x),3) for x in xgb_proba]
        val_batch_df['fraud'] = validation_y
        predicted_val_df = predicted_val_df.append(val_batch_df)
        counter +=1
        

    return predicted_val_df

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras_metrics import binary_average_recall, binary_recall, binary_precision
import pandas as pd
import numpy as np

def keras_model():
    model = Sequential()
    model.add(Dense(15, input_shape=(4,),activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model

def load_trained_model():
    model = keras_model()
    model.load_weights(weights_path)
    return model

Using TensorFlow backend.
